In [1]:
#Definindo finais de semana
mes = "07"
lista_fds = ['06','07','13','14','20','21','27','28','31']

# Biblioteca e Funções

In [2]:
####################################-BIBLIOTECAS-####################################

#manipulação de dados
import pandas as pd
import numpy as np

#string e data
import unidecode #limpeza
import re #regex

#tempo
import time
from time import sleep

#web scraping
import requests
from bs4 import BeautifulSoup

#automação
from selenium import webdriver #navegador
from selenium.webdriver.common.keys import Keys #comandos do teclado
from selenium.webdriver.common.by import By 
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait #esperar
from selenium.webdriver.support.expected_conditions import (visibility_of, 
                                                           staleness_of, 
                                                           invisibility_of_element)
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pyautogui #

import warnings #Warning inuteis
from pandas.errors import SettingWithCopyWarning
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

#manipulação do diretorio
import os

In [3]:
# FUNÇÕES XPATH
def xpath_element(xpath):
    return browser.find_element("xpath", xpath)

def xpath_click(xpath):
    try:
        xpath_element(xpath).click()
    except:
        sleep(2)
        xpath_element(xpath).click()
    
def xpath_send_keys(xpath, texto):
    xpath_element(xpath).send_keys(texto)
    
def xpath_clear(xpath):
    xpath_element(xpath).clear()

def existencia(caminho):
    elementos = browser.find_elements(By.XPATH, caminho)
    return bool(elementos)

In [4]:
#FUNÇÕES
def count_freq(data_frame, coluna):
    freq = df[coluna].value_counts()
    perc = df[coluna].value_counts(normalize = True)*100
    tabela = pd.DataFrame({"Freq":freq, "Perc":perc})
    return display(tabela)

def salvando_nome_arquivo(nome_arquivo, dataframe):
    if os.path.exists(nome_arquivo): #O nome_arquivo existe
        df1 = pd.read_excel(nome_arquivo)
        len_antigo = len(dataframe)
        
        dataframe = pd.concat([df1, dataframe]).reset_index(drop=True)
        dataframe = dataframe.drop_duplicates(subset=['chat'])
        
        print(len(df1), " += ", len_antigo, "=>", len(dataframe))
        
        dataframe.to_excel(nome_arquivo, index=False)
    
    else: #O nome_arquivo não existe
        dataframe.to_excel(nome_arquivo, index=False)
        print(len(dataframe)) 

# ===============================================

# Ligando navegador

In [5]:
#browser.quit(); sleep(5)
browser = webdriver.Firefox()

#fazendo login
browser.get('https://www.pegaplantao.com.br/login')
sleep(1)
xpath_send_keys('//*[@id="MainContent_LoginUser_UserName"]', 'gabriel_s_anjos@yahoo.com') #e-mail
xpath_send_keys('//*[@id="MainContent_LoginUser_Password"]', 'Gabriel2022!') #senha
xpath_click('//*[@id="MainContent_LoginUser_btnLogin"]') #login

In [6]:
browser.get('https://www.pegaplantao.com.br/EscalaMensal/old')

In [7]:
#click setor/profissional
xpath_click('//*[@id="s2id_autogen1"]/a/span[1]')

#click [profissional
xpath_click('//*[@id="select2-drop"]/ul/li[2]/div')

# Capturando setores:

In [9]:
#click criar novo plantão
def criar_plantao():
    xpath_click('//*[@id="shift-monthly"]/div[2]/table/tbody/tr[1]/td[4]/button')
criar_plantao()

def esperar_elemento_aparecer(elemento, tempo):
    sleep(0.5)
    WebDriverWait(browser, tempo).until(visibility_of(xpath_element(elemento)))

sleep(2)
esperar_elemento_aparecer('//*[@id="service-ckb-list-all-owner"]', 10)
print('apareceu')

def esperar_elemento_desaparecer(elemento, tempo):
    sleep(0.5)
    WebDriverWait(browser, tempo).until_not(visibility_of(xpath_element(elemento)))

apareceu


In [10]:
#CAPTURANDO DADOS DO UNIDADE E SETOR
xpath_click('//*[@id="s2id_ddlServiceSector"]/a/span[1]')
setor_html = xpath_element('//*[@id="select2-drop"]/ul').get_attribute('outerHTML')
soup = BeautifulSoup(setor_html, 'html.parser')

lista_unidade = []
lista_unidade_count = []
lista_plt = []
lista_plt_count = []

count_unidade = 1

for i in soup.find_all(class_="select2-results-dept-0 select2-result select2-result-unselectable select2-result-with-children"):
    #print('\n', i.find(class_='select2-result-label').get_text(), '====== \n')
    count_plt = 1
    for t in i.find_all(class_='select2-results-dept-1 select2-result select2-result-selectable'): 
        #print(t.get_text())
        lista_plt.append(t.get_text())
        lista_plt_count.append(count_plt)
        count_plt += 1

        lista_unidade.append(i.find(class_='select2-result-label').get_text())
        lista_unidade_count.append(count_unidade)
    count_unidade += 1

scrap_unidades = pd.DataFrame({'unidade':lista_unidade, 'momento':lista_plt,
                               'unidade_count':lista_unidade_count, 'plt_count':lista_plt_count})

scrap_unidades['momento'] = scrap_unidades['momento'].apply(lambda x: x.replace('  ',' '))

scrap_unidades['momento'] = scrap_unidades['momento'].apply(lambda x: unidecode.unidecode(x.replace('  ',' ').upper().strip()))
scrap_unidades['unidade'] = scrap_unidades['unidade'].apply(lambda x: unidecode.unidecode(x.replace('  ',' ').upper().strip()))
print('OK')

OK


In [11]:
#CARREGANDO OS DADOS PARA SUBIR NO PEGA PLANTÃO
df = pd.read_excel('df_gabs.xlsx') # df_gabs # df_continuar
df = df.sort_values(['nome', 'data']).reset_index(drop=True)

df['setor'][(df['setor'] == 'FDS - C.CIRURGICO ROTINA') & (df['unidade'] == 'SANTA ISABEL')] = 'FDS - MANHA - C.CIRURGICO'

print(len(df))
df.sample(3)

76


,chat,dia,nome,mensagem,controle,data,unidade,extra,tipo,ENTRADA,SAIDA,DURAÇÃO,PERIODO,periodo,setor,sem_ou_fds,unidade2
57,[08/07/2024 14:03:30] Raphael Moura Britto: 08...,08/07/2024,RAPHAEL MOURA BRITTO,08/07 MORUMBI - BRITTO ENTRADA 06:00 ( DES...,DESLOCADO 6H,08/07/2024,MORUMBI,NaN,ENTRADA,06:00,07:00,01:00,6h,1,SEMANA - 6HS,semana,"['MORUMBI', 'ITAIM']"
72,[02/07/2024 06:35:23] Vitor Oliveira André: 02...,02/07/2024,VITOR OLIVEIRA ANDRE,02/07 VNS VITOR ENTRADA 06:30,NaN,02/07/2024,VNS,NaN,ENTRADA,06:30,07:00,00:30,6h,1,SEMANA - 6HS,semana,VNS
60,[16/07/2024 10:53:08] Ricardo Ivo de Lima: 16/...,16/07/2024,RICARDO IVO DE LIMA,16/07 - MORUMBI - RICARDO IVO ENTRADA - ...,DESLOCADO 6H,16/07/2024,MORUMBI,NaN,ENTRADA,06:00,07:00,01:00,6h,1,SEMANA - 6HS,semana,"['MORUMBI', 'ITAIM']"


## Pareando Setores

In [12]:
setores = pd.read_excel("LocaiseSetores.xlsx")
print(setores.shape)

setores = setores.dropna() #
setores = setores.reset_index(drop=True)

for coluna in ['Local', 'Setor', 'Mudou para:']:
    setores[coluna] = setores[coluna].apply(lambda x: unidecode.unidecode(x).upper().strip())
    setores[coluna] = setores[coluna].apply(lambda x: x.replace('  ', ' '))
    setores[coluna] = setores[coluna].apply(lambda x: x.replace('  ', ' '))

print(setores.shape)
setores.sample(5)

(230, 5)
(217, 5)


,Código,Local,Setor,Situação,Mudou para:
160,9.0,OSASCO,FDS - D3,Habilitado,16 - FDS - D3 (R)
90,4.0,ITAIM,SEMANA - NOTURNO - DISTANCIA,Habilitado,08 - NOTURNA - DISTANCIA
45,4.0,ITAIM,SEMANA - TARDE - C.CIRURGICO,Habilitado,04.1 - SEMANA - CENTRO CIRURGICO
153,11.0,SANTA ISABEL,FDS - SAIDA,Habilitado,14 - FDS - SAIDA
114,1.0,ANALIA,FDS - 6HS,Habilitado,11 - FDS - 06 HS


In [13]:
df['setor_antigo'] = df['setor']

for unidade_df in df['unidade'].unique():
    for setor_df in df['setor'][df['unidade'] == unidade_df].unique():
    
        setores_filter = setores[(setores['Local'] == unidade_df) & ((setores['Setor'] == setor_df) | (setores['Mudou para:'] == setor_df))]
        if len(setores_filter) >= 1:
            #print(unidade_df, '|', setor_df, '===========================')
            #print('encontrado')

            df['setor'][(df['unidade'] == unidade_df) & (df['setor'] == setor_df)] = setores['Mudou para:'][setores_filter.index.values[0]]

            pass
        else:
            print(unidade_df, '|', setor_df, '=!=!=!=!=!=!=!=!=!=!=!=!=!=!')
            print('não encontrado')

In [14]:
#TESTANDO COMPATIBILIDADE DOS DADOS CAPTURADOS E DADOS CARREGADOS
for X in range(0, len(df)):
    if len(scrap_unidades.loc[(scrap_unidades.unidade == df['unidade'][X]) & 
                                 (scrap_unidades.momento == df['setor'][X])]) < 1:
        print(X)
        print(df['unidade'][X], '-', df['setor'][X])
        #print('index', ind)
        #print(scrap_unidades['unidade'][ind], '-', scrap_unidades['momento'][ind])
        #print('codigos:', unidade, setorzin)
        

print("="*20, "TUDO OK", "="*20)

==================== TUDO OK ====================


In [15]:
print(df['setor'].unique())

df['setor_inicio'] = np.nan
for i in range(len(df)):
    corte = df['setor'][i].find('-')
    texto = df['setor'][i][:corte].strip()
    df['setor_inicio'][i] = texto

print(df['setor_inicio'].unique())

['14 - FDS - SAIDA' '04.1 - SEMANA - CENTRO CIRURGICO'
 '04 - SEMANA - CENTRO CIRURGICO' '01 - NOTURNA - ENTRADA ANTES DAS 07HS'
 '13 - FDS - CENTRO CIRURGICO' '11 - FDS - 06 HS']
['14' '04.1' '04' '01' '13' '11']


# Funções Selenium

In [55]:
def subir_pg():
    browser.execute_script("window.scrollTo(0, 0);")

In [56]:
def reset_selecao():
    #xpath_click('/html')#; sleep(0.3)
    #xpath_click('/html')

    posicao_antiga = pyautogui.position() 
    pyautogui.click(x=66, y=158); sleep(0.3)
    pyautogui.click(x=66, y=158)
    pyautogui.moveTo(posicao_antiga) 


def fechar_caixa_de_selecao():
    # Localiza um elemento que é seguro para clicar (por exemplo, o corpo da página)
    elemento_seguro = browser.find_element(By.TAG_NAME, 'body')
    # Cria uma ação para clicar no elemento seguro
    ActionChains(browser).move_to_element(elemento_seguro).click().perform()
def reset_selecao1():
    #fechar_caixa_de_selecao(); sleep(0.3)
    #fechar_caixa_de_selecao()

    posicao_antiga = pyautogui.position() 
    pyautogui.click(x=66, y=158); sleep(0.3)
    pyautogui.click(x=66, y=158)
    pyautogui.moveTo(posicao_antiga)

def reset_selecao2():
    posicao_antiga = pyautogui.position() 
    pyautogui.click(x=66, y=158); sleep(0.3)
    pyautogui.click(x=66, y=158)
    pyautogui.moveTo(posicao_antiga) 
reset_selecao2()

In [57]:
#selecionando setor
def setor(unidade, setor):
    #print('base:', unidade, '-', setor)
    xpath_click('//*[@id="s2id_ddlServiceSector"]/a/span[1]')
    ind = scrap_unidades.loc[(scrap_unidades.unidade == unidade) & 
                             (scrap_unidades.momento == setor)].index.values[0]
    unidade_sel = scrap_unidades['unidade_count'][ind]
    setor_sel = scrap_unidades['plt_count'][ind]
    xpath_click('//*[@id="select2-drop"]/ul/li['+str(unidade_sel)+']/ul/li['+str(setor_sel)+']/div')   
setor("SANTA ISABEL", '04 - SEMANA - CENTRO CIRURGICO')

#inserir e selecionar profissional
def nome_func(nome_profissional):
    #listar todos
    xpath_click('//*[@id="service-ckb-list-all-owner"]')
    
    xpath_click('//*[@id="service-ckb-list-all-owner"]')
    xpath_click('//*[@id="service-ckb-list-all-owner"]')
    sleep(0.3)
    xpath_click('//*[@id="s2id_ddlProfessionalA"]/a/span[1]')
    xpath_send_keys('//*[@id="select2-drop"]/div/input', nome_profissional)
    #Dando enter
    xpath_send_keys('//*[@id="select2-drop"]/div/input', Keys.RETURN)
nome_func('OTTO MITTERMAYER')

#inserir horario e duração
def hora(inicio, duracao):
    #Inicio do plantão
    xpath_clear('//*[@id="txtServiceStartTime"]')
    xpath_send_keys('//*[@id="txtServiceStartTime"]', inicio)
    sleep(0.3)
    xpath_clear('//*[@id="txtServiceDuration"]')
    xpath_send_keys('//*[@id="txtServiceDuration"]', duracao)
hora('13:30', '05:00')

In [58]:
#inserir data
def data_func(data_selecionada):
    data_selecionada = str(data_selecionada)
    i_chato2 = str(5) #4 e 5
    i_chato = str(3) #2 e 3

    xpath_clear('/html/body/div['+i_chato2+']/div/div[2]/div[2]/div[5]/div[1]/div[2]/div/div[2]/div[1]/div['+i_chato+']/input[1]')
    #xpath_clear('/html/body/div[4]/div/div[2]/div[2]/div[5]/div[1]/div[2]/div/div[2]/div[1]/div[3]/input[2]')
    sleep(0.3)
    
    xpath_send_keys('/html/body/div['+i_chato2+']/div/div[2]/div[2]/div[5]/div[1]/div[2]/div/div[2]/div[1]/div['+i_chato+']/input[1]', data_selecionada)
    #xpath_send_keys('/html/body/div[4]/div/div[2]/div[2]/div[5]/div[1]/div[2]/div/div[2]/div[1]/div[3]/input[2]', data_selecionada)
    reset_selecao2()
data_func("15/07/2024")

In [20]:
def tipo_registro_func(i):
    xpath_click('//*[@id="s2id_add-service-service-types"]/a/span[1]')
    sleep(0.5)

    xpaths = [
        '/html/body/div[12]/div/input',
        '/html/body/div[11]/div/input',
        '/html/body/div[10]/div/input',
        '/html/body/div[9]/div/input',
        '/html/body/div[8]/div/input',
        '/html/body/div[7]/div/input',
        '/html/body/div[6]/div/input',
    ]

    for xpath in xpaths:
        try:
            xpath_send_keys(xpath, i)
            sleep(1)
            xpath_para_clicar = xpath[:xpath.find(']')+1] + '/ul/li/div'
            xpath_click(xpath_para_clicar)
            return  # Sai da função se encontrar um xpath que funcione
        except:
            continue  # Tenta o próximo xpath se ocorrer uma exceção
    
    print('erro')  # Imprime 'erro' se nenhum xpath funcionar

tipo_registro_func('VOLUNTÁRIO')

In [59]:
#inserindo pontuação
def pontuacao(pontos): # NÃO SE ALTERA MAIS
    #xpath_clear('//*[@id="txtServiceValue"]')
    #xpath_send_keys('//*[@id="txtServiceValue"]', pontos) #Pontos
    pass

pontuacao("666")

#selecionar tipo de plantão e ajustando tempo
def tipo_do_registro(duracao, sem_ou_fds, tipo_registro):
    if tipo_registro == 'extra':
        print('EXTRA')
        sleep(0.5)
        reset_selecao2()
        sleep(1)
        tipo_registro_func('VOLUNTÁRIO') #escrevendo
        if int(duracao[:2]) >= 5 and sem_ou_fds == "semana":
            pontuacao(200)
            pontuacao(200)
            print('duração >= 5 => 200 pontos')
        else:
            pass
    elif tipo_registro == 'DESLOCADO':
        pass
        #print('DESLOCADO')
        #sleep(1)
        #reset_selecao2()
        
        #xpath_click('//*[@id="s2id_add-service-service-types"]/a/span[1]')
        #xpath_send_keys('/html/body/div[11]/div/input', 'VOLUNTÁRIO') #escrevendo
        #xpath_click('/html/body/div[11]/ul/li/div') #clicando no primeiro
        #if int(duracao[:2]) >= 5 and sem_ou_fds == "semana":
        #    pontuacao(100)
        #    pontuacao(100)
        #    print('duração >5 -> 100 pontos')
        #else:
        #    pass
        #sleep(1)
    else:
        if int(duracao[:2]) >= 5 and sem_ou_fds == "semana":
            pontuacao(100)
            pontuacao(100)
            print('duração >5 -> 100 pontos')
        else:
            pass
        sleep(1)

#tipo_do_registro("05:00", "semana", "extra")
#tipo_do_registro(df['DURAÇÃO'][h], df['sem_ou_fds'][h], df['extra'][h])

In [22]:
def apagando_registro():
    chatao = str(5) #4 ou 5
    #Apagando
    browser.find_element("xpath", '/html/body/div['+chatao+']/div/div[1]/div[2]/button[1]').click()
    #Clicando em continuar
    sleep(0.5)
    browser.find_element("xpath", '/html/body/div['+chatao+']/div/div[1]/div[3]/button[2]').click()
#apagando_registro()

In [42]:
def insistindo_no_xpath(xpathzinho):
    xpath_click(xpathzinho)
    try:
        esperar_elemento_desaparecer(xpathzinho, 10)
    except:
        xpath_click(xpathzinho)

#########################################################

#salvando plantão
def salvar_plantao():
    xpathzinho = '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]'
    xpath_click(xpathzinho)
#salvar_plantao()

#fechando plantão
def fechar():
    xpathzinho = '//*[@id="modal-add-service"]/div/div[1]/button'
    xpath_click(xpathzinho)
#fechar()

#atualizando plantão
def atualizando_plantao():
    #xpathzinho = '/html/body/div[4]/div/div[1]/div[2]/button[5]'
    xpathzinho = '/html/body/div[5]/div/div[1]/div[2]/button[5]'
    xpath_click(xpathzinho)
#atualizando_plantao()

In [24]:
fechar()

# Função Tabela

In [25]:
######### ######### FUNÇÃO DE CAPTURA ######### ######### 

#pacotes para manipular data
from datetime import date
from datetime import time
from datetime import datetime
from datetime import timedelta

#capturando registros
unidades = [
            'ADMINISTRACAO','ALPHA','ANALIA',
            'CENTRAL DO TATUAPE',
            'HOSPITAIS EXTERNOS','HOSPITAL PARA TESTE',
            'ITAIM',
            'JABAQUARA',
            'MATERNIDADE','MORUMBI',
            'OSASCO',
            'SANTA ISABEL',
            'VILLA','VNS'
            ]

def funcao():
    lista_tabela = []
    for i in range(1, 6): #passando por cada semana[1 a 5]
    ##print('semana:', i, "=="*20)
        for t in range(1, 8): #passando por cada dia[1 a 7] dentro da semana
            reg = xpath_element('/html/body/form/main/div[2]/div[2]/table/tbody/tr['+str(i)+']/td['+str(t)+']').text
            content = reg.splitlines() #deixando as palavras em um item dentro de uma lista
            content = [unidecode.unidecode(i.strip().upper()) for i in content] #tratando cada palavra

            dia = content[0] #dia do registro
            del(content[0]) #deletando o dia, ja que foi coletado
            del(content[-1]) #deletando residuo do wepscraping

            if len(content) != 0: #ignorando dias em branco (sem registro) ¹
                #print('dia:', t)

                ##print('dia:', dia)
                #print(content)

                count_hora = 0 #count para capturar a hora de entrada e saida, count=1(entrada) | count=2(saida)
                lista_control = []
                for y in content: #passando um for sobre cada item da lista
                    if y in unidades: # capturando a unidade e dando inicio a novo registro novo
                        #print('================unidade:', y)
                        unidade = y
                    elif y[2] == ':': #reconhecendo se o item é hora (HH*:*mm)
                        count_hora += 1
                        if count_hora == 1: #capturando entrada
                            #print('hora entrada:', y)
                            lista_control.append(y)
                        elif count_hora == 2: #capturando saida
                            #print('hora saida:', y)
                            lista_control.append(y)
                            ##print('lista teste', lista_control)
                            lista_tabela.append(lista_control)
                            lista_control = []
                            count_hora = 0
                    else: #capturando dia, unidade, e periodo
                        #print('.')
                        #print('periodo:', y)
                        lista_control.append(dia)
                        lista_control.append(unidade)
                        lista_control.append(y)
                        lista_control.append(i)
                        lista_control.append(t) #dia semanal

                ##print('='*20)    

            else:
                continue #ignorando dias em branco (sem registro) ²
    
    tabela = pd.DataFrame(lista_tabela, columns = ['Dia','Unidade','Periodo','Semana','Dia Semana','Entrada','Saida'])
    
    #limpeza da tabela
    #########3
    tabela['Dia'] = tabela['Dia'].apply(lambda x: int(x))

    #recolhendo os dias do mes passado que ainda aparecem na primeira semana
    dias_mes_passado = list(tabela[(tabela['Dia'] > 20) & (tabela['Semana'] == 1)].index) 
    tabela.drop(dias_mes_passado, axis=0, inplace=True) #excluindo
    tabela.reset_index(drop=True, inplace=True) #resetando index

    #Inserindo a ordem dos registros
    tabela['ordem'] = ''

    for i in tabela['Dia'].unique():
        for t in range(0, len(tabela[tabela['Dia'] == i].index)):
            tabela['ordem'][tabela[tabela['Dia'] == i].index[t]] = t+1

    #Inserindo coluna de data        
    tabela['data'] = tabela['Dia'].apply(lambda x: str(x))

    for i in range(len(tabela)):
        if len(tabela['data'][i]) == 1:
            tabela['data'][i] = "0" + tabela['data'][i]

        tabela['data'][i] = tabela['data'][i] + "/" + mes + "/2024" 

    tabela = tabela.query("Unidade != 'ADMINISTRACAO'")
    tabela = tabela[~((tabela['Semana'] >= 5) & (tabela['Dia'] < 10))]
    tabela.reset_index(drop=True, inplace=True) #resetando index
    
    tabela['PERIODO'] = np.nan
    for i in range(0, len(tabela)):
        if int(tabela['Entrada'][i][:2]) <= 6:
            tabela['PERIODO'][i] = '6h'
        elif int(tabela['Entrada'][i][:2]) >= 7 and int(tabela['Entrada'][i][:2]) <= 12:
            tabela['PERIODO'][i] = 'manha'
        elif int(tabela['Entrada'][i][:2]) >= 13 and int(tabela['Entrada'][i][:2]) <= 18:
            tabela['PERIODO'][i] = 'tarde'
        else:
            tabela['PERIODO'][i] = 'noturno saida'

    tabela['periodo'] = np.nan
    tabela['periodo'][tabela['PERIODO'] == '6h'] = 1
    tabela['periodo'][tabela['PERIODO'] == 'manha'] = 2
    tabela['periodo'][tabela['PERIODO'] == 'tarde'] = 3
    tabela['periodo'][tabela['PERIODO'] == 'noturno saida'] = 4
    
    return tabela
    
tabela = funcao()
tabela.head()

,Dia,Unidade,Periodo,Semana,Dia Semana,Entrada,Saida,ordem,data,PERIODO,periodo


# CAPTURANDO DADOS

## Nomes

In [26]:
def diferenca_horas(horario_inicial, horario_final):
    formato = "%H:%M"
    horario_inicial = datetime.strptime(horario_inicial, formato)
    horario_final = datetime.strptime(horario_final, formato)
    diferenca = horario_final - horario_inicial
    if diferenca.days < 0:
        diferenca = horario_inicial - horario_final
        return diferenca.total_seconds() / 60 * (-1)
    else:
        return diferenca.total_seconds() / 60

In [27]:
# Selecionar nome
def xpath_estranho(i):
    xpaths = [
        '/html/body/div[12]/div/input',
        '/html/body/div[11]/div/input',
        '/html/body/div[10]/div/input',
        '/html/body/div[9]/div/input',
        '/html/body/div[8]/div/input',
        '/html/body/div[7]/div/input',
        '/html/body/div[6]/div/input',
    ]
    
    for xpath in xpaths:
        try:
            xpath_send_keys(xpath, i)
            return  # Sai da função se encontrar um xpath que funcione
        except:
            continue  # Tenta o próximo xpath se ocorrer uma exceção
    
    print('erro')  # Imprime 'erro' se nenhum xpath funcionar
#xpath_estranho(i)

In [28]:
xpath_click('//*[@id="s2id_ddlProfessionals"]/a') #barra de nome
sleep(2)

html = browser.find_element("xpath", '//*[@id="select2-drop"]/ul').get_attribute('outerHTML') #
soup_nome = BeautifulSoup(html, 'html.parser')
#soup_nome
NOMES = []
for i in soup_nome.find_all(class_ ="select2-results-dept-0 select2-result select2-result-selectable"):
    #print(i.get_text())
    NOMES.append(i.get_text())
#del(NOMES[0])

reset_selecao()
display(NOMES)
print(len(NOMES))

['ALESSANDRE LUIZ BRAGA JUNIOR',
 'ALEXANDRE AMARAL LUCAS',
 'ALEXANDRE BERNARDINO',
 'ALEXSANDER TAKASHI HAMADA',
 'ALINE FRANCHINI GOULART LIMA',
 'AMANDA KIYOKO SOUZA KOSSAKA',
 'ANA CLARA MOURÃO BARRETO',
 'ANA GABRIELA GEWEHR VALE SALMEN',
 'ANA JUNG DOCKO',
 'ANDRÉ AUGUSTO BELLINI',
 'ANDRÉ DA COSTA AGUIAR VENTURA',
 'ANDRÉ LUIS XAVIER FRANCO',
 'ANDRÉ MORTARI PLÁ GIL',
 'ANDRÉIA LOUREIRO MORI',
 'ANNA BEATRIZ ARANHA TREVISAN',
 'ANTONIO ALVARO DIAS LIMA DA CUNHA NETO',
 'ARTHUR OLIVEIRA FACCHINI',
 'ARTHUR RODRIGUES TORRELIO',
 'AUGUSTO DE CASTRO SANTOS QUEIROZ',
 'BEATRIZ ANTIPOU DOS SANTOS',
 'BIANCA PRISCILA BUNIOTTI',
 'BRIAN SOSSAI PACHECO',
 'BRUCE YE MAN CHOW',
 'BRUNA DAIBERT DANESIN',
 'BRUNO FERRAZ DE CAMPOS BARONE',
 'BRUNO STORNIOLI TAGUCHI LEMOS',
 'CAIO TULIO DUARTE',
 'CAMILA CHIARANTANO HARRISON',
 'CAMILA RESENDE DE PAULA',
 'CARLOS AUGUSTO TREVISAN',
 'CARLOS EDUARDO CORRÊA GRILO',
 'CARLOS EDUARDO GONDIM OLIVEIRA',
 'CAROLINE FIRMINO LEMES DA SILVA',
 'CLAUDIO

240


In [32]:
len(df['nome'].unique())

53

In [30]:
count_control = 0

tabelona = pd.DataFrame(columns=['Nome'] + list(funcao().columns))
tabelona

,Nome,Dia,Unidade,Periodo,Semana,Dia Semana,Entrada,Saida,ordem,data,PERIODO,periodo


In [31]:
count_nome = 0 

#for NOME in NOMES[count_control:]:
for NOME in df['nome'].unique():

    if count_control > 0:
        if NOMES[count_control] != NOMES[count_control-1] or count_nome == 0:
            print(NOMES[count_control]) 
            #print('\n', '='*20, NOMES[count_control], '='*20, '\n')
            print(count_control, '|', len(NOMES)-count_control)

    xpath_click('//*[@id="s2id_ddlProfessionals"]/a/span[1]'); sleep(1)
    xpath_estranho(NOME)
    xpath_click('//*[@id="select2-drop"]/ul/li/div')

    WebDriverWait(browser, 30).until_not(visibility_of(xpath_element('//*[@id="loader"]')))
    
    tabela = funcao()
    tabela['Nome'] = NOME
    tabelona = pd.concat([tabelona, tabela])
    count_control += 1

ALEXANDRE AMARAL LUCAS
1 | 239
ALEXANDRE BERNARDINO
2 | 238
ALEXSANDER TAKASHI HAMADA
3 | 237
ALINE FRANCHINI GOULART LIMA
4 | 236
AMANDA KIYOKO SOUZA KOSSAKA
5 | 235
ANA CLARA MOURÃO BARRETO
6 | 234
ANA GABRIELA GEWEHR VALE SALMEN
7 | 233
ANA JUNG DOCKO
8 | 232
ANDRÉ AUGUSTO BELLINI
9 | 231
ANDRÉ DA COSTA AGUIAR VENTURA
10 | 230
ANDRÉ LUIS XAVIER FRANCO
11 | 229
ANDRÉ MORTARI PLÁ GIL
12 | 228
ANDRÉIA LOUREIRO MORI
13 | 227
ANNA BEATRIZ ARANHA TREVISAN
14 | 226
ANTONIO ALVARO DIAS LIMA DA CUNHA NETO
15 | 225
ARTHUR OLIVEIRA FACCHINI
16 | 224
ARTHUR RODRIGUES TORRELIO
17 | 223
AUGUSTO DE CASTRO SANTOS QUEIROZ
18 | 222
BEATRIZ ANTIPOU DOS SANTOS
19 | 221
BIANCA PRISCILA BUNIOTTI
20 | 220
BRIAN SOSSAI PACHECO
21 | 219
BRUCE YE MAN CHOW
22 | 218
BRUNA DAIBERT DANESIN
23 | 217
BRUNO FERRAZ DE CAMPOS BARONE
24 | 216
BRUNO STORNIOLI TAGUCHI LEMOS
25 | 215
CAIO TULIO DUARTE
26 | 214
CAMILA CHIARANTANO HARRISON
27 | 213
CAMILA RESENDE DE PAULA
28 | 212
CARLOS AUGUSTO TREVISAN
29 | 211
CARLOS ED

In [33]:
tabelona.reset_index(drop=True, inplace=True)

print(tabelona['Periodo'].unique())

tabelona['setor_inicio'] = np.nan
for i in range(len(tabelona)):
    corte = tabelona['Periodo'][i].find('-')
    texto = tabelona['Periodo'][i][:corte].strip()
    tabelona['setor_inicio'][i] = str(texto)

print(tabelona['setor_inicio'].unique())

['04 - SEMANA - CENTRO CIRURGICO' '04.1 - SEMANA - CENTRO CIRURGICO'
 '01 - NOTURNA - ENTRADA ANTES DAS 07HS' '02 - SEMANA - AV.PRE ANESTESICA'
 '07 - NOTURNA - SAIDA APOS 19HS' '11 - FDS - 06 HS'
 '13 - FDS - CENTRO CIRURGICO' '19 - FDS - D1' '16 - FDS - D3 (R)'
 '08 - NOTURNA - DISTANCIA' '09 - NOTURNA - PLANTAO' '18 - FDS - D2 (R)'
 '20 - FDS - PLANTAO' '02.1 - SEMANA - AV.PRE ANESTESICA'
 '15 - FDS - D3 (B)' '06 - SEMANA - ENDOSCOPIA' '14 - FDS - SAIDA'
 '04.3 - SEMANA - COBERTURA EXTRA' '04.4 - SEMANA - COBERTURA EXTRA'
 '12 - FDS - AV.PRE ANESTESICA']
['04' '04.1' '01' '02' '07' '11' '13' '19' '16' '08' '09' '18' '20' '02.1'
 '15' '06' '14' '04.3' '04.4' '12']


In [34]:
tabelona['Nome'] = tabelona['Nome'].apply(lambda nome: unidecode.unidecode(nome).upper().strip().replace("  ", " "))
tabelona_backup = tabelona.copy

print(tabelona.shape)
tabelona.sample(5)

(2140, 13)


,Nome,Dia,Unidade,Periodo,Semana,Dia Semana,Entrada,Saida,ordem,data,PERIODO,periodo,setor_inicio
1360,PEDRO AUGUSTO TAVARES DALL AGLIO,26,ANALIA,04.1 - SEMANA - CENTRO CIRURGICO,4,5,13:00,19:00,2,26/07/2024,tarde,3.0,04.1
1627,RAPHAEL MOURA BRITTO,19,ITAIM,06 - SEMANA - ENDOSCOPIA,3,5,07:00,13:00,1,19/07/2024,manha,2.0,06
45,ANA CLARA MOURAO BARRETO,12,ITAIM,04.1 - SEMANA - CENTRO CIRURGICO,2,5,13:00,16:00,1,12/07/2024,tarde,3.0,04.1
1715,RICARDO IVO DE LIMA,18,MORUMBI,04 - SEMANA - CENTRO CIRURGICO,3,4,07:00,13:00,1,18/07/2024,manha,2.0,04
1738,RODRIGO MARTINS PEREIRA,6,VILLA,14 - FDS - SAIDA,1,6,13:00,16:30,2,06/07/2024,tarde,3.0,14


# =======================================================================

# Diminuindo Tempos & Deslocado

In [35]:
from selenium.webdriver.common.action_chains import ActionChains
action_chains = ActionChains(browser)

## Pareamento

In [36]:
df['oque_fazer_com_o_registro'] = np.nan

for i in range(len(df)):

    nome = df['nome'][i]
    data = df['data'][i]
    periodo = df['PERIODO'][i]

    tabela = tabelona[(tabelona['Nome'] == nome) & 
                    (tabelona['data'] == data) &
                    (tabelona['PERIODO'] == periodo)]
    if len(tabela) == 0:
        df['oque_fazer_com_o_registro'][i] = 'criar plantao'

    elif len(tabela) == 1: # Verificando se os horarios batem
        if df['ENTRADA'][i] != tabela['Entrada'][tabela.index[0]] or df['SAIDA'][i] != tabela['Saida'][tabela.index[0]]:
            df['oque_fazer_com_o_registro'][i] = 'alterar horario'
        else:
            df['oque_fazer_com_o_registro'][i] = 'nada' # OK

    elif len(tabela) >= 2:
        df['oque_fazer_com_o_registro'][i] = 'apagar duplicata'
    else:

        print("WTF ?")
        break

count_freq(df, 'oque_fazer_com_o_registro')

,Freq,Perc
oque_fazer_com_o_registro,,
criar plantao,53,69.736842
nada,9,11.842105
alterar horario,9,11.842105
apagar duplicata,5,6.578947


In [37]:
df_filter = df[df['oque_fazer_com_o_registro'] == 'nada']
salvando_nome_arquivo('Feitos.xlsx', df_filter)

df_filter = df[df['oque_fazer_com_o_registro'] == 'alterar horario']
os.remove("df_gabs.xlsx")
salvando_nome_arquivo('df_gabs.xlsx', df_filter)

df_filter = df[df['oque_fazer_com_o_registro'] == 'criar plantao']
salvando_nome_arquivo('df_gabs2.xlsx', df_filter)

df_filter = df[df['oque_fazer_com_o_registro'] == 'apagar duplicata']
salvando_nome_arquivo('df_giov.xlsx', df_filter)

df = df[df['oque_fazer_com_o_registro'] == 'alterar horario'].reset_index(drop=True)
df_backup = df.copy()

2848  +=  9 => 2857
9
53
188  +=  5 => 193


In [38]:
count = 0
print(df_backup.shape)
df_backup.head()

(9, 20)


,chat,dia,nome,mensagem,controle,data,unidade,extra,tipo,ENTRADA,SAIDA,DURAÇÃO,PERIODO,periodo,setor,sem_ou_fds,unidade2,setor_antigo,setor_inicio,oque_fazer_com_o_registro
0,[05/07/2024 20:28:13] Bruna Daibert Danesin: 0...,05/07/2024,BRUNA DAIBERT DANESIN,05/07 ANÁLIA BRUNA SAÍDA 18:00,NaN,05/07/2024,ANALIA,NaN,SAIDA,13:00,18:00,05:00,tarde,3,04.1 - SEMANA - CENTRO CIRURGICO,semana,ANALIA,SEMANA - TARDE - C.CIRURGICO,04.1,alterar horario
1,[11/07/2024 17:10:19] Camila Resende de Paula:...,11/07/2024,CAMILA RESENDE DE PAULA,11/07 ITAIM C. RESENDE SAÍDA 17:10,NaN,11/07/2024,ITAIM,NaN,SAIDA,13:00,17:30,04:30,tarde,3,04.1 - SEMANA - CENTRO CIRURGICO,semana,ITAIM,SEMANA - TARDE - C.CIRURGICO,04.1,alterar horario
2,[11/07/2024 19:16:04] Felipe de Oliveira Nunes...,11/07/2024,FELIPE DE OLIVEIRA NUNES,11/07 ANÁLIA - NUNES SAÍDA 18:20,NaN,11/07/2024,ANALIA,NaN,SAIDA,13:00,18:30,05:30,tarde,3,04.1 - SEMANA - CENTRO CIRURGICO,semana,ANALIA,SEMANA - TARDE - C.CIRURGICO,04.1,alterar horario
3,[04/07/2024 19:49:21] Flavio Hideki Oshika: 04...,04/07/2024,FLAVIO HIDEKI OSHIKA,04/07 JABAQUARA - FLÁVIO SAÍDA 18:20,NaN,04/07/2024,JABAQUARA,NaN,SAIDA,13:00,18:30,05:30,tarde,3,04.1 - SEMANA - CENTRO CIRURGICO,semana,JABAQUARA,SEMANA - TARDE - C.CIRURGICO,04.1,alterar horario
4,[08/07/2024 09:07:04] Leonardo Barbosa Santos:...,08/07/2024,LEONARDO BARBOSA SANTOS,08/07 JABAQUARA L.SANTOS SAIDA 09:05,NaN,08/07/2024,JABAQUARA,NaN,SAIDA,07:00,09:30,02:30,manha,2,04 - SEMANA - CENTRO CIRURGICO,semana,JABAQUARA,SEMANA - MANHA - C.CIRURGICO,04,alterar horario


# Execução

In [44]:
############################-CODIGO-############################
#Para cada profissional na lista
df = df_backup[count:]

for i in df['nome'].unique():
    print('='*20, i, '='*20)
    df_nome = df[df['nome'] == i]
    df_nome = df_nome.reset_index(drop=True)

    ###########################################
    # MUDANDO O PROFISSIONAL ESCOLHIDO
    subir_pg(); sleep(0.5)
    xpath_click('//*[@id="s2id_ddlProfessionals"]/a/span[1]'); sleep(1)
    xpath_estranho(i)
    xpath_click('//*[@id="select2-drop"]/ul/li/div')
    ###########################################
    
    # esperando carregamento do site
    WebDriverWait(browser, 30).until_not(visibility_of(xpath_element('//*[@id="loader"]')))
    sleep(2)
    # Capturando calendario de registros do individuo
    tabela = funcao(); sleep(0.3)
    
    # Para cada registro
    for t in range(len(df_nome)):
        ##########################################################################################################
        print(df_nome['mensagem'][t])

        '''
        # Analisandos os registros no **dia**
        dia_sel = tabela[tabela['data'] == df_nome['data'][t]] # & (tabela['Periodo'] == df_nome["setor"][t])]
        # Encurtando entrada - quando entrou atrasado
        if df_nome['PERIODO'][t] == 'manha' and df_nome['sem_ou_fds'][t] == 'fds':
            dia_sel = dia_sel[dia_sel['Entrada'] == '07:00']
        else:
            dia_sel = dia_sel[dia_sel['PERIODO'] == df_nome["PERIODO"][t]]
        '''

        ####################################################
        dia_sel = tabela[(tabela['data'] == df_nome['data'][t]) & (tabela['PERIODO'] == df_nome['PERIODO'][t])]
        ####################################################

        dia_sel.reset_index(drop=True, inplace=True)        
        ##########################################################################################################

        if len(dia_sel) == 0:
            df_backup['controle'][count] = "PROB"; print(count, "-> PROB")
        elif len(dia_sel) == 2:
            # abrindo/selecionando periodo conforme localização
            try:
                xpath_localizacao = '//*[@id="shift-monthly"]/div[2]/table/tbody/tr['+str(dia_sel['Semana'][0])+']/td['+str(dia_sel['Dia Semana'][0])+']/ul['+str(dia_sel['ordem'][0])+']/li'
                esperar_elemento_aparecer(xpath_localizacao, 5)
                xpath_click(xpath_localizacao)
            except:
                xpath_localizacao = '//*[@id="shift-monthly"]/div[2]/table/tbody/tr['+str(dia_sel['Semana'][0])+']/td['+str(dia_sel['Dia Semana'][0])+']/ul/li['+str(dia_sel['ordem'][0])+']'
                esperar_elemento_aparecer(xpath_localizacao, 5)
                xpath_click(xpath_localizacao)
            # Esperar componentes carregarem
            WebDriverWait(browser, 10).until(visibility_of(xpath_element('//*[@id="service-ckb-list-all-owner"]')))
            sleep(0.3)
            
            #apagando_registro(); sleep(0.3)
            tipo_registro_func('DISPENSADO'); sleep(0.5); atualizando_plantao()
            
            df_backup['controle'][count] = "OK"; print(count, "-> OK")
            WebDriverWait(browser, 10).until(visibility_of(xpath_element('//*[@id="s2id_ddlProfessionals"]/a/span[1]')))
            sleep(1)
        else:
            #abrindo/selecionando periodo conforme localização
            try:
                xpath_localizacao = '//*[@id="shift-monthly"]/div[2]/table/tbody/tr['+str(dia_sel['Semana'][0])+']/td['+str(dia_sel['Dia Semana'][0])+']/ul['+str(dia_sel['ordem'][0])+']/li'
                esperar_elemento_aparecer(xpath_localizacao, 5)
                xpath_click(xpath_localizacao)
            except:
                xpath_localizacao = '//*[@id="shift-monthly"]/div[2]/table/tbody/tr['+str(dia_sel['Semana'][0])+']/td['+str(dia_sel['Dia Semana'][0])+']/ul/li['+str(dia_sel['ordem'][0])+']'
                esperar_elemento_aparecer(xpath_localizacao, 5)
                xpath_click(xpath_localizacao)
            #Esperar componentes carregarem
            WebDriverWait(browser, 10).until(visibility_of(xpath_element('//*[@id="service-ckb-list-all-owner"]')))
            sleep(1)
            hora(df_nome['ENTRADA'][t], df_nome['DURAÇÃO'][t])
            if dia_sel['Unidade'][0] != df_nome['unidade'][t]:
                setor(df_nome['unidade'][t], df_nome['setor'][t])
            else:
                pass
            
            tipo_do_registro(df_nome['DURAÇÃO'][t], df_nome['sem_ou_fds'][t], df_nome['extra'][t])
            sleep(0.3)
            reset_selecao2()
            #salvar plantão
            sleep(0.3)
            atualizando_plantao(); atualizando_plantao(); sleep(0.3)
            df_backup['controle'][count] = "OK"; print(count, "-> OK")
            WebDriverWait(browser, 10).until(visibility_of(xpath_element('//*[@id="s2id_ddlProfessionals"]/a/span[1]')))
            sleep(1)
            
        count += 1 
        
        if count % 25 == 0:
            print('='*80, '\n', df_backup['controle'].value_counts(), '\n'); df_backup.to_excel("df_gabs.xlsx", index=False)
        else:
            if t != len(df_nome)-1:
                print('='*20)
            
print("="*50, 'TERMINOU', "="*50)
print('Total  ', len(df_backup))
print('='*80, df_backup['controle'].value_counts(), '='*80); df_backup.to_excel("df_gabs.xlsx", index=False)

==================== FELIPE DE OLIVEIRA NUNES ====================
11/07 ANÁLIA  -  NUNES SAÍDA 18:20
duração >5 -> 100 pontos
2 -> OK
==================== FLAVIO HIDEKI OSHIKA ====================
04/07 JABAQUARA  -  FLÁVIO SAÍDA 18:20
duração >5 -> 100 pontos
3 -> OK
==================== LEONARDO BARBOSA SANTOS ====================
08/07 JABAQUARA L.SANTOS SAIDA 09:05
4 -> OK
==================== NATAN SOARES SILVEIRA ====================
07/07 ITAIM  -  NATAN  -  ENTRADA 07:00  ( PLANTÃO ) 
5 -> OK
==================== RAFAEL QUEIROZ ARAUJO ====================
04/07 MORUMBI  -  RAFAEL ENTRADA 04:25
6 -> OK
==================== SAULLO QUEIROZ SILVEIRA ====================
07/07 ITAIM SAULLO SAÍDA 10:50
7 -> OK
==================== VITOR MACIEL DE SOUSA PINTO ====================
12/07 ANÁLIA  -  MACIEL SAÍDA 09:45
8 -> OK
================================================== TERMINOU ==================================================
Total   9
==========================================

In [43]:
df_backup.loc[int(count), 'controle'] = "OK"
count += 1
print(df_backup['nome'][count])
print(count)

FELIPE DE OLIVEIRA NUNES
2


In [ ]:
count += 1

In [ ]:
count += 1

In [ ]:
df['nome'][df['nome'] == 'ICARO MOURA DE SOUSA'] = 'ICARO DE MOURA SOUSA'

## Salvando

In [45]:
df_corrigidos = df_backup[df_backup['controle'] == "OK"]
print("probs ok:", len(df_corrigidos))

df_prob = df_backup[df_backup['controle'] != "OK"]
print("probs:", len(df_prob))

###############################################################

os.remove("df_gabs.xlsx")
salvando_nome_arquivo('Feitos.xlsx', df_corrigidos)
print('.')
salvando_nome_arquivo('df_gabs2.xlsx', df_prob)

probs ok: 9
probs: 0
2857  +=  9 => 2866
.
53  +=  0 => 53


# ============================

# Plantões Normais

In [46]:
df = pd.read_excel('df_gabs2.xlsx')
df = df.sort_values(['nome', 'data']).reset_index(drop=True)
print(df.shape)
display(df.head())

(53, 20)


,chat,dia,nome,mensagem,controle,data,unidade,extra,tipo,ENTRADA,SAIDA,DURAÇÃO,PERIODO,periodo,setor,sem_ou_fds,unidade2,setor_antigo,setor_inicio,oque_fazer_com_o_registro
0,[06/07/2024 13:18:56] Amanda Kiyoko Souza Koss...,06/07/2024,AMANDA KIYOKO SOUZA KOSSAKA,06/07 OSASCO - AMANDA SAÍDA 13:10 ( DESLOCA...,DESLOCADO,06/07/2024,OSASCO,NaN,SAIDA,13:00,13:30,00:30,tarde,3,14 - FDS - SAIDA,fds,"['OSASCO', 'ITAIM']",FDS - SAIDA,14.0,criar plantao
1,[06/07/2024 07:42:21] André da Costa Aguiar Ve...,06/07/2024,ANDRE DA COSTA AGUIAR VENTURA,05/07 ITAIM VENTURA ENTRADA 06:30 ( DESLOCADO...,DESLOCADO 6H,05/07/2024,ITAIM,NaN,ENTRADA,06:30,07:00,00:30,6h,1,01 - NOTURNA - ENTRADA ANTES DAS 07HS,semana,"['ITAIM', 'VNS']",SEMANA - 6HS,1.0,criar plantao
2,[02/07/2024 18:03:45] Anna Beatriz Aranha Trev...,02/07/2024,ANNA BEATRIZ ARANHA TREVISAN,02/07 VILLA - ANNA BEATRIZ SAÍDA 18:00,NaN,02/07/2024,VILLA,NaN,SAIDA,13:00,18:00,05:00,tarde,3,04.1 - SEMANA - CENTRO CIRURGICO,semana,VILLA,SEMANA - TARDE - C.CIRURGICO,4.1,criar plantao
3,[03/07/2024 23:19:20] Anna Beatriz Aranha Trev...,03/07/2024,ANNA BEATRIZ ARANHA TREVISAN,03/07 VILLA - ANNA BEATRIZ SAÍDA 18:00,NaN,03/07/2024,VILLA,NaN,SAIDA,13:00,18:00,05:00,tarde,3,04.1 - SEMANA - CENTRO CIRURGICO,semana,VILLA,SEMANA - TARDE - C.CIRURGICO,4.1,criar plantao
4,[09/07/2024 16:15:58] Anna Beatriz Aranha Trev...,09/07/2024,ANNA BEATRIZ ARANHA TREVISAN,09/07 VILLA - ANNA BEATRIZ SAÍDA 16:10,NaN,09/07/2024,VILLA,NaN,SAIDA,13:00,16:30,03:30,tarde,3,04.1 - SEMANA - CENTRO CIRURGICO,semana,VILLA,SEMANA - TARDE - C.CIRURGICO,4.1,criar plantao


In [47]:
len(df)

53

In [48]:
H=0

# Execução

In [62]:
count = 0
for h in range(H, len(df)):
    #####################################################################################################################
    if h > 0:
        if df['nome'][h] != df['nome'][h-1] or count == 0:
            print('='*20, df['nome'][h], '='*20)
            count += 1
    #####################################################################################################################
    #try:
    print('[',h,'] |','dia:', df['data'][h],'| mensagem:', df['mensagem'][h])
    
    #click criar novo plantão
    criar_plantao()
    
    #SETOR - UNIDADE
    #esta disponivel o botão de setor ?
    esperar_elemento_aparecer('//*[@id="s2id_ddlServiceSector"]/a/span[1]', 10)
    #Selecionando Setor
    setor(df['unidade'][h], df['setor'][h])
    
    #PROFISSIONAL
    #esta disponivel o botão de profissional
    esperar_elemento_aparecer('//*[@id="service-ckb-list-all-owner"]', 10)
    nome_func(df['nome'][h])
    sleep(1)

    #DATA DO PLANTÃO
    data_func(df['data'][h])
    
    #INICIO DO PLANTÃO -- DURAÇÃO DO PLANTÃO
    hora(df['ENTRADA'][h], df['DURAÇÃO'][h])
    sleep(1)
        
    #TIPO
    tipo_do_registro(df['DURAÇÃO'][h], df['sem_ou_fds'][h], df['extra'][h])
    
    #SALVAR PLANTÃO
    sleep(1)
    salvar_plantao()
    salvar_plantao()
    salvar_plantao()
    sleep(4)

    #VERIFICANDO SE HOUVE ERRO - HÁ PLANTÃO NO MESMO HORARIO
    chatao = 5 #6 5 4
    if xpath_element('/html/body/div['+str(chatao)+']/div/div[2]/div[1]').text != "":
        fechar()
        df.loc[int(h), 'controle'] = "PROBS"; print("PROB")
    else:
        df.loc[int(h), 'controle'] = "NORM"; print("NORM")
    #####################################################################################################################
    '''except Exception as e:
        print(f"Erro ao processar o arquivo: {e}")
        if existencia('//*[@id="modal-add-service"]/div/div[1]/button'):
            fechar()
            print('*'*50)
            df.loc[int(h), 'controle'] = "ERRO_REPETIR"; print("ERRO_REPETIR")
            print('*'*50)
        else:
            print('-VERIFICAR-'*10)
            break'''
            
    #####################################################################################################################
    # praticamente um # finally
    WebDriverWait(browser, 10).until_not(visibility_of(xpath_element('//*[@id="modal-add-service"]/div/div[1]/button')))
    H += 1
    #####################################################################################################################
    if H % 25 == 0:
        print('='*80, '\n', df['controle'].value_counts(), '\n', '='*80); df.to_excel("df_gabs2.xlsx", index=False)
    if df['nome'][h] == df['nome'][h+1]:
        print('='*10)
        
print("="*20, 'TERMINOU', "="*20)
print('Total  ', len(df))
print('='*80, '\n', df['controle'].value_counts(), '\n', '='*80); df.to_excel("df_gabs2.xlsx", index=False)

==================== TERMINOU ====================
Total   53
 controle
PROBS    52
NORM      1
Name: count, dtype: int64 


In [ ]:
#df.loc[int(h), 'controle'] = "NORM"; print("NORM")
df.loc[int(h), 'controle'] = "PROBS"; print("PROB")
H += 1

In [ ]:
df['nome'][df['nome'] == '~ ICARO MOURA'] = 'icaro de MOURA SOUSA'.upper()

## Salvando

In [63]:
print(df['controle'].value_counts())
print(df['controle'].value_counts(normalize=True)*100)

count_freq(df, 'controle')

controle
PROBS    52
NORM      1
Name: count, dtype: int64
controle
PROBS    98.113208
NORM      1.886792
Name: proportion, dtype: float64


,Freq,Perc
controle,,
PROBS,52,98.113208
NORM,1,1.886792


In [64]:
df_corrigidos = df[df['controle'] == "NORM"]
print("probs ok:", len(df_corrigidos))

df_prob = df[df['controle'] != "NORM"]
print("probs:", len(df_prob))

os.remove("df_gabs2.xlsx")
###############################################################

salvando_nome_arquivo('Feitos.xlsx', df_corrigidos)
print('.')
salvando_nome_arquivo('df_giov2.xlsx', df_prob)

probs ok: 1
probs: 52
2866  +=  1 => 2867
.
52


# =======================

# Dispensados

In [65]:
df_backup = pd.read_excel('df_dispensados.xlsx')
df_backup = df_backup.drop_duplicates(subset=['chat'])
df_backup = df_backup.sort_values(['nome','data']).reset_index(drop=True)

print(df_backup.shape)
df_backup.head()

(1, 7)


,chat,dia,nome,mensagem,controle,data,hora
0,[13/07/2024 07:20:50] Marcos Bastos dos Santos...,13/07/2024,MARCOS BASTOS DOS SANTOS,13/07 ANÁLIA - BASTOS - DISPENSADO,OK,13/07/2024,NaN


In [66]:
count = 0
df_backup['controle'] = np.nan

# Execução

def apagando_registro():
    chatao = str(5) #4 ou 5
    #Apagando
    browser.find_element("xpath", '/html/body/div['+chatao+']/div/div[1]/div[2]/button[1]').click()
    #Clicando em continuar
    sleep(0.5)
    browser.find_element("xpath", '/html/body/div['+chatao+']/div/div[1]/div[3]/button[2]').click()
apagando_registro()

In [67]:
len(df_backup)

1

In [68]:
#Para cada profissional na lista
df = df_backup[count:]

for i in df['nome'].unique():
    print('='*20, i, '='*20)
    df_nome = df[df['nome'] == i]
    df_nome.reset_index(drop=True, inplace=True)
    
    xpath_click('//*[@id="s2id_ddlProfessionals"]/a/span[1]'); sleep(1)
    xpath_estranho(i)
    xpath_click('//*[@id="select2-drop"]/ul/li/div')

    #esperando carregamento do site
    WebDriverWait(browser, 30).until_not(visibility_of(xpath_element('//*[@id="loader"]')))

    tabela = funcao()    
    sleep(2)
    
    #Para cada registro na lista_nome
    for t in range(len(df_nome)):
        print(df_nome['mensagem'][t])
        dia_sel = tabela[tabela['data'] == df_nome['data'][t]]
        dia_sel.reset_index(drop=True, inplace=True)
        
        if len(dia_sel) == 0:
            print(count, "-> Ja apagado")
            print(df_nome['chat'][t] + " - " + df_nome['data'][t])
            df_backup['controle'][count] = "OK"
        else:
            for r in range(len(dia_sel)-1, -1, -1):
                print('PRINCIPAL')
                #abrindo/selecionando periodo conforme localização
                dia_ordem = r
                xpath_textao = '//*[@id="shift-monthly"]/div[2]/table/tbody/tr['+str(dia_sel['Semana'][0])+']/td['+str(dia_sel['Dia Semana'][0])+']/ul['+str(dia_sel['ordem'][dia_ordem])+']/li'
                xpath_click(xpath_textao)
                #Esperar componentes carregarem
                WebDriverWait(browser, 10).until(visibility_of(xpath_element('//*[@id="service-ckb-list-all-owner"]')))
                sleep(2)
                reset_selecao2()
                tipo_registro_func('DISPENSADO')
                atualizando_plantao()
                reset_selecao2()
                atualizando_plantao()
                
                #esperando carregamento do site para prosseguir
                WebDriverWait(browser, 20).until_not(visibility_of(xpath_element('//*[@id="modal-add-service"]/div/div[1]/button')))
            
            tabela = funcao() #atualizando tabela de registros
            sleep(2)
            print(count, '-> OK')
            print(df_nome['chat'][t] + " - " + df_nome['data'][t])
            df_backup['controle'][count] = "OK"
            
        count += 1 
        if count % 25 == 0:
            print('='*80, '\n', df_backup['controle'].value_counts(), '\n', '='*80); df_backup.to_excel("df_dispensados.xlsx", index=False)
        else:
            if t != len(df_nome)-1:
                print('='*20)
    
print('='*25, 'TERMINOU', '='*25)
print('Total  ', len(df_backup))
print('='*80, df_backup['controle'].value_counts(), '='*80); df_backup.to_excel("df_dispensados.xlsx", index=False)

==================== MARCOS BASTOS DOS SANTOS ====================
13/07 ANÁLIA  -  BASTOS  - DISPENSADO
PRINCIPAL
0 -> OK
[13/07/2024 07:20:50] Marcos Bastos dos Santos: 13/07 Anália - Bastos -Dispensado - 13/07/2024
========================= TERMINOU =========================
Total   1
================================================================================ controle
OK    1
Name: count, dtype: int64 ================================================================================


In [ ]:
count += 1 

In [ ]:
df_backup['controle'][count] = "OK"
count += 1

In [ ]:
df_backup['nome'][df_backup['nome'] == "ICARO MOURA DE SOUSA"] = "ICARO DE MOURA SOUSA"

## Salvando

In [69]:
df_corrigidos = df_backup[df_backup['controle'] == "OK"]
print("probs ok:", len(df_corrigidos))

df_prob = df_backup[df_backup['controle'] != "OK"]
print("probs:", len(df_prob))

################################################################

os.remove("df_dispensados.xlsx")
salvando_nome_arquivo('Feitos.xlsx', df_corrigidos)
print('.')
salvando_nome_arquivo('df_giov.xlsx', df_prob)

probs ok: 1
probs: 0
2867  +=  1 => 2868
.
193  +=  0 => 193


# =============================

# Encerrando Automação

In [70]:
browser.quit()

# ==============================

# Residuos Finais

In [71]:
df_feito = pd.read_excel("Feitos.xlsx")
print(len(df_feito))

df_prob0 = pd.read_excel("df_giov.xlsx")
print(len(df_prob0))
os.remove("df_giov.xlsx")

df_prob01 = pd.read_excel("df_giov2.xlsx")
print(len(df_prob01))
os.remove("df_giov2.xlsx")

df_prob = pd.concat([df_prob0, df_prob01])
df_prob = df_prob.drop_duplicates(subset=['chat'])
salvando_nome_arquivo('df_prob.xlsx', df_prob)

print(len(df_prob))

2868
193
52
52
52


In [74]:
total = len(df_feito) + len(df_prob)

percent = len(df_feito) / total
percent = round(percent*100, 2)
percent = str(percent) + "%"
percent = "("+percent+")"
print("Registros App:", len(df_feito), percent)

percent = len(df_prob) / total
percent = round(percent*100, 2)
percent = str(percent) + "%"
percent = "("+percent+")"
print("Registros Giovanelli", len(df_prob), percent)
print("#"*30, "\n")

for nome in sorted(list(df_prob['nome'].unique())):
    print(nome, "="*10)
    selecao = df_prob.loc[df_prob['nome']==nome]
    selecao.reset_index(drop=True, inplace=True)
    
    for i in range(len(selecao)):
        print(selecao["mensagem"][i])
    
    print("\n")

Registros App: 2868 (92.13%)
Registros Giovanelli 245 (7.87%)
############################## 

ALESSANDRE LUIZ BRAGA JUNIOR ==========
05/07 ANALIA  -  ALESSANDRE ENTRADA 07:00  ( DISTANCIA ) 


ALEXANDRE AMARAL LUCAS ==========
01/07 MORUMBI  -  ALEXANDRE AMARAL ENTRADA  -  07:00  ( DISTÂNCIA ) 
01/07 MORUMBI ALEXANDRE AMARAL SAÍDA 23:50  ( DISTÂNCIA ) 
15/07 MORUMBI  -  ALEXANDRE AMARAL ENTRADA 07:00  ( DISTÂNCIA ) 


ALEXANDRE BERNARDINO ==========
03/07  -  ANÁLIA  -  ALEXANDRE  -  SAÍDA 01:40
14/07  -  ANÁLIA  -  ALEXANDRE  -  ENTRADA 08:20  ( D3 ) 
14/07  -  ANÁLIA  -  ALEXANDRE  -  SAÍDA 13:35  ( D3 ) 


ALEXSANDER TAKASHI HAMADA ==========
13/07 VILLA ALEX SAÍDA 1:35


AMANDA KIYOKO SOUZA KOSSAKA ==========
05/07 MORUMBI  -  AMANDA SAÍDA 23:45
06/07 OSASCO  -  AMANDA SAÍDA 13:10  ( DESLOCADO ITAIM ) 


ANA CLARA MOURAO BARRETO ==========
02/07 HCT CLARA SAÍDA 14:25
04/07 OSASCO CLARA SAÍDA 18:50


ANA JUNG DOCKO ==========
07/07 MORUMBI ANA SAÍDA 16:15  ( D2 ) 


ANDRE DA COSTA

In [75]:
os.remove("df_prob.xlsx")
df_prob.to_excel("df_prob_enviado_1907.xlsx", index=False)

# =================================

# Teste

In [ ]:
######### ######### FUNÇÃO DE CAPTURA ######### ######### 
#capturando registros
unidades = [
            'ADMINISTRACAO','ALPHA','ANALIA',
            'CENTRAL DO TATUAPE',
            'HOSPITAIS EXTERNOS','HOSPITAL PARA TESTE',
            'ITAIM',
            'JABAQUARA',
            'MATERNIDADE','MORUMBI',
            'OSASCO',
            'SANTA ISABEL',
            'VILLA','VNS'
            ]

lista_tabela = []
for i in range(1, 6): #passando por cada semana[1 a 5]
##print('semana:', i, "=="*20)
    for t in range(1, 8): #passando por cada dia[1 a 7] dentro da semana
        reg = xpath_element('/html/body/form/main/div[2]/div[2]/table/tbody/tr['+str(i)+']/td['+str(t)+']').text
        content = reg.splitlines() #deixando as palavras em um item dentro de uma lista
        content = [unidecode.unidecode(i.strip().upper()) for i in content] #tratando cada palavra

        dia = content[0] #dia do registro
        del(content[0]) #deletando o dia, ja que foi coletado
        del(content[-1]) #deletando residuo do wepscraping

        if len(content) != 0: #ignorando dias em branco (sem registro) ¹
            #print('dia:', t)

            ##print('dia:', dia)
            #print(content)

            count_hora = 0 #count para capturar a hora de entrada e saida, count=1(entrada) | count=2(saida)
            lista_control = []
            for y in content: #passando um for sobre cada item da lista
                if y in unidades: # capturando a unidade e dando inicio a novo registro novo
                    #print('================unidade:', y)
                    unidade = y
                elif y[2] == ':': #reconhecendo se o item é hora (HH*:*mm)
                    count_hora += 1
                    if count_hora == 1: #capturando entrada
                        #print('hora entrada:', y)
                        lista_control.append(y)
                    elif count_hora == 2: #capturando saida
                        #print('hora saida:', y)
                        lista_control.append(y)
                        ##print('lista teste', lista_control)
                        lista_tabela.append(lista_control)
                        lista_control = []
                        count_hora = 0
                else: #capturando dia, unidade, e periodo
                    #print('.')
                    #print('periodo:', y)
                    lista_control.append(dia)
                    lista_control.append(unidade)
                    lista_control.append(y)
                    lista_control.append(i)
                    lista_control.append(t) #dia semanal

            ##print('='*20)    

        else:
            continue #ignorando dias em branco (sem registro) ²

tabela = pd.DataFrame(lista_tabela, columns = ['Dia','Unidade','Periodo','Semana','Dia Semana','Entrada','Saida'])


In [ ]:
for i in range(len(lista_tabela)):
    print(lista_tabela[i])